<a href="https://colab.research.google.com/github/Kadar5692/Settyl-Project/blob/main/Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Load the dataset
data_file_path = '/content/drive/MyDrive/Colab Notebooks/dataset.json'
df = pd.read_json(data_file_path)

# Check for missing values
print("Missing values in the dataset:")
print(df.isnull().sum())

# One-hot encode 'externalStatus'
encoder = OneHotEncoder(drop='first', sparse=False)
external_status_encoded = pd.DataFrame(encoder.fit_transform(df[['externalStatus']]))

# Encode 'internalStatus' using LabelEncoder
label_encoder = LabelEncoder()
internal_status_encoded = label_encoder.fit_transform(df['internalStatus'])

# Combine encoded features with other numerical features
X = pd.concat([external_status_encoded, df.drop(columns=['externalStatus', 'internalStatus'])], axis=1).values
y = internal_status_encoded

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Save the trained model
model.save('/content/drive/MyDrive/Colab Notebooks/trained_model.h5')

# Define FastAPI app
!pip install fastapi
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Define input data model
class InputData(BaseModel):
    external_status: str

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/trained_model.h5')

# Define endpoint for making predictions
@app.post("/predict/")
async def predict(input_data: InputData):
    # Preprocess input data
    external_status = np.array([input_data.external_status])
    external_status_encoded = encoder.transform(external_status.reshape(-1, 1))

    # Make predictions
    predictions = model.predict(external_status_encoded)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Map predicted class to label (you may need to adjust this based on your label encoding)
    labels = {i: label for i, label in enumerate(label_encoder.classes_)}
    predicted_label = labels.get(predicted_class, 'Unknown')

    return {"predicted_internal_status": predicted_label}


Missing values in the dataset:
externalStatus    0
internalStatus    0
dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/20
31/31 [==============================] - 1s 11ms/step - loss: 2.5278 - accuracy: 0.3971 - val_loss: 2.3713 - val_accuracy: 0.4204
Epoch 2/20
31/31 [==============================] - 0s 4ms/step - loss: 2.1145 - accuracy: 0.4852 - val_loss: 1.9268 - val_accuracy: 0.5592
Epoch 3/20
31/31 [==============================] - 0s 4ms/step - loss: 1.6153 - accuracy: 0.6643 - val_loss: 1.5227 - val_accuracy: 0.6776
Epoch 4/20
31/31 [==============================] - 0s 4ms/step - loss: 1.2089 - accuracy: 0.7451 - val_loss: 1.1931 - val_accuracy: 0.6776
Epoch 5/20
31/31 [==============================] - 0s 3ms/step - loss: 0.9041 - accuracy: 0.7472 - val_loss: 0.9521 - val_accuracy: 0.6776
Epoch 6/20
31/31 [==============================] - 0s 3ms/step - loss: 0.7017 - accuracy: 0.7564 - val_loss: 0.7881 - val_accuracy: 0.7347
Epoch 7/20
31/31 [==============================] - 0s 3ms/step - loss: 0.5605 - accuracy: 0.8373 - val_loss: 0.6548 - val_accuracy: 0.8327
Epoch 8/20
31/31 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.3 MB/s eta 0:00:00


In [ ]:
pip install fastapi uvicorn pyngrok nest-asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.post("/predict/")
async def predict(data: dict):
    # Perform prediction logic here
    return {"message": "Received data", "data": data}


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Make sure to set the auth token before using ngrok
ngrok.set_auth_token("2esBSCdqijEL6CcQdcMSQnZftVa_3aNhL3yWpDyboS41dKDuW")

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)

# Get public URL
public_url = ngrok_tunnel.public_url
print("Public URL:", public_url)

# Run the FastAPI app using uvicorn
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [591]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://457f-34-82-4-155.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [591]


KeyboardInterrupt: 